# **DeBERTa**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from datasets import Dataset
from transformers import (
    DebertaTokenizer,
    DebertaForSequenceClassification,
    Trainer,
    TrainingArguments,
)
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [ ]:
# ----------------
# Load dataset
# ----------------

path_file = "/content/df_ndf.csv"
df = pd.read_csv(path_file, sep="\t", encoding="utf-8")
df.head()

texts = df["texts"].astype(str).tolist()
labels = df["labels"].values



# Codifica etichette
encoder = LabelEncoder()
df["labels"] = encoder.fit_transform(df["labels"])

# Split train/test
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df["texts"].tolist(),
    df["labels"].tolist(),
    test_size=0.2,
    stratify=df["labels"],
    random_state=42,
)

In [ ]:
# -----------------------------
# Tokenizzazione con DeBERTa
# -----------------------------
tokenizer = DebertaTokenizer.from_pretrained("microsoft/deberta-base")

max_len = 256  # coerente con fine-tuning BERT nel paper
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=max_len)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=max_len)


In [ ]:
# -----------------------------------
# Conversione in Dataset Hugging Face
# -----------------------------------
train_dataset = Dataset.from_dict(
    {
        "input_ids": train_encodings["input_ids"],
        "attention_mask": train_encodings["attention_mask"],
        "labels": train_labels,
    }
)

test_dataset = Dataset.from_dict(
    {
        "input_ids": test_encodings["input_ids"],
        "attention_mask": test_encodings["attention_mask"],
        "labels": test_labels,
    }
)

In [ ]:
# ---------------------------
# Caricamento modello DeBERTa
# ---------------------------
model = DebertaForSequenceClassification.from_pretrained(
    "microsoft/deberta-base",
    num_labels=2  # classificazione binaria
)

In [ ]:
# -----------------------
# Metriche di valutazione
# -----------------------
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average="weighted"
    )
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

In [ ]:
# --------------------------------------
# Impostazioni di training (Trainer API)
# --------------------------------------
training_args = TrainingArguments(
    output_dir="./deberta_fake_news",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    save_total_limit=2,
)

In [ ]:
# -------------------------
# Addestramento con Trainer
# -------------------------
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
# -----------
# Valutazione
# -----------
metrics = trainer.evaluate()
print("\n=== Risultati Finali (DeBERTa) ===")
for k, v in metrics.items():
    print(f"{k}: {v:.4f}")